In [29]:
import torch
import torch.nn as nn

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [12]:
abc = np.random.rand(4,1,1)

In [13]:
abc

array([[[0.73018325]],

       [[0.56910205]],

       [[0.94644541]],

       [[0.90381443]]])

In [14]:
abc[1].shape

(1, 1)

In [15]:
abc.shape

(4, 1, 1)

In [18]:
# List of tensors
tensor_list = [torch.randn(3, 4), torch.randn(3, 4), torch.randn(3, 4)]

# Convert list of tensors to a single tensor using torch.stack
stacked_tensor = torch.stack(tensor_list, dim=1)

In [19]:
stacked_tensor

tensor([[[-1.5104,  1.3448,  0.1017, -0.2000],
         [ 0.5491, -1.0699,  0.9874,  0.3279],
         [ 0.9167, -1.8975, -0.3729,  0.7786]],

        [[ 3.4593, -0.8323, -0.1711,  0.0885],
         [ 1.0751,  2.0139, -1.3565, -0.0302],
         [-0.6259,  0.2220,  1.3950,  0.7904]],

        [[ 1.1181, -1.5235,  0.9631,  0.1366],
         [ 1.5289, -0.3659,  0.9838,  0.8275],
         [-1.2917, -1.9094, -1.0863, -1.9231]]])

In [26]:
import numpy as np

# Define matrices W and X
W = np.array([[1, 2],
              [3, 4]])

X = np.array([[5, 6],
              [7, 8]])

# Matrix multiplication W * X
W_times_X = np.dot(W, X)

# Matrix multiplication X * W^T
X_times_W_transpose = W.T@X

print("W * X:")
print(W_times_X)
print("\nX * W^T:")
print(X_times_W_transpose)


W * X:
[[19 22]
 [43 50]]

X * W^T:
[[26 30]
 [38 44]]


In [36]:
abc = torch.rand(1,34) * 2 -1
abc

tensor([[-0.5889, -0.9657,  0.8762, -0.9311, -0.8791,  0.2524, -0.0533,  0.2300,
         -0.8445,  0.5449, -0.0423,  0.1963, -0.6934,  0.7806, -0.7166,  0.2464,
         -0.4169, -0.2981, -0.8897,  0.9948,  0.6422,  0.6820,  0.2948, -0.9628,
          0.1384,  0.5283,  0.6956,  0.6767,  0.6397,  0.6770,  0.3466, -0.5976,
         -0.5648,  0.3531]])

In [37]:
m = nn.ReLU()
m(abc)

tensor([[0.0000, 0.0000, 0.8762, 0.0000, 0.0000, 0.2524, 0.0000, 0.2300, 0.0000,
         0.5449, 0.0000, 0.1963, 0.0000, 0.7806, 0.0000, 0.2464, 0.0000, 0.0000,
         0.0000, 0.9948, 0.6422, 0.6820, 0.2948, 0.0000, 0.1384, 0.5283, 0.6956,
         0.6767, 0.6397, 0.6770, 0.3466, 0.0000, 0.0000, 0.3531]])